This is a rudimentary version of the signal processing algorithm

In [2]:
import mne
import yasa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='white', font_scale=1.2)
from scipy.signal import welch

We will need some way of reading in the data, and splitting it by 30 s epochs.

In [3]:
#Read-in raw data
filename='sub-02_mne_raw.fif'
raw= mne.io.read_raw_fif(filename, preload=True, verbose=0)
#Note: this will not work in real-time- I'm unsure how that will work
times= raw.times
tmax= times.size #Probably isn't in seconds
epoch_num=tmax//30
#for epoch in epoch_num:
    #raw.save('epoch'+epoch+'.fif', tmin=30*(epoch-1), tmax=30*epoch)
#Issue: this will save a separate file for each epoch, which is not efficient and we should avoid
print(epoch_num)
raw.save('epoch1.fif', tmin=0, tmax=30, overwrite=True)
#Some variable may be needed to keep track of time
#At t=30, the raw data will be split off into its own array and a new array will be created for the next epoch
    #This will probably involve the use of an iterator variable
    #Maybe use 2D array?
#Filter the signal!!

9800


<ipython-input-3-4c418f0137b9>:11: RuntimeWarning: This filename (C:\Users\User\Desktop\CruX Project\notebooks\epoch1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw.save('epoch1.fif', tmin=0, tmax=30, overwrite=True)


In [8]:
#Calculate bandpower or FFT power
epoch1=mne.io.read_raw_fif('epoch1.fif', preload=True, verbose=0)
epoch1.pick_types(eeg=True)
data=epoch1._data*1e6 #Changing units from V to microvolts
sf = epoch1.info['sfreq'] #Set this to whatever the actual sampling frequency is

bp=yasa.bandpower(data, sf=sf) #channel names haven't been set- should be fine since we only have one channel
#print(bp)
alpha=bp.at['CHAN000', 'Alpha']
#print(alpha)
beta=bp.at['CHAN000', 'Beta']
#print(beta)
#need theta
#Slow wave detection
slow=yasa.sw_detect(data, sf=sf)
#print(slow.summary(grp_chan=True,aggfunc='mean'))
slow_num=(slow.summary(grp_chan=True,aggfunc='mean')).at['CHAN000', 'Count'] #May need to add paramters for this function
#print(slow_num)
spindles=yasa.spindles_detect(data, sf=sf)
#print(spindles)
if spindles== None:
    spindles_num=0
    print(spindles_num)
else:
    spindles_shape=(spindles.summary()).size()
    spindles_num=spindles_shape(1) #In theory, spindles_shape will be the dimensions, and this will grab the number of rows
    print(spindles_num)
    #This side of the loop still needs to be tested

<ipython-input-8-44e19cda8c68>:2: RuntimeWarning: This filename (epoch1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  epoch1=mne.io.read_raw_fif('epoch1.fif', preload=True, verbose=0)
28-Feb-21 20:25:55 | WARNING | No SW were found in channel CHAN003.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN000.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN001.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN002.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN003.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN004.
28-Feb-21 20:25:55 | WARNING | No spindle were found in channel CHAN005.
28-Feb-21 20:25:55 | WARNING | No spindles were found in data. Returning None.


0


In [29]:
slow_threshold=1 #Placeholder value
stages=np.empty(5, dtype='S5') #Check code properly later- this will need to set "stages" to have the same number of elements as epohcs
i=0
#Wakefulness detection
if (alpha+beta>0.5):
    if theta>0.25:
        stages[i]='R'
    else:
        stages[i]='W'
else:
    if slow_num>slow_threshold: #Will need to set a threshold for # of slow waves in N3
        stages[i]='N3'
    elseif spindles_num>spin_threshold:
        stages[i]='N2'
    else:
        stages[i]='N1'
print(stages)
#Issues: detection doesn't seem to be correct, formatting on array- this will probably work better with integers

[b'N3' b'\x01\x00\x00s' b'\x00\x00e' b'\x00:' b'1\x005\x00:']
